# Precomputed Perlin Noise Vector Field

A circular path in noise space drives a time-dependent 2D field. The workflow now splits into a simulation phase that precomputes vector samples and a render phase that reuses the cached data.

In [ ]:
%load_ext manim.utils.ipython_magic

In [ ]:
from __future__ import annotations

import math
from dataclasses import dataclass
from pathlib import Path
from typing import Tuple, cast

import numpy as np
from tqdm.auto import tqdm
from manim import (
    Scene,
    ValueTracker,
    Arrow,
    VGroup,
    RIGHT,
    color_gradient,
    ManimColor,
    FadeIn,
    FadeOut,
    smooth,
    BLUE_E,
    BLUE_C,
    GREEN_B,
    YELLOW_C,
    RED_C,
)

# Minimal Perlin sampler so the notebook has no extra runtime dependencies.
def _fade(t: np.ndarray | float) -> np.ndarray | float:
    return 6 * t**5 - 15 * t**4 + 10 * t**3


def _lerp(a: np.ndarray | float, b: np.ndarray | float, t: np.ndarray | float) -> np.ndarray | float:
    return a + t * (b - a)


@dataclass
class PerlinSampler:
    seed: int = 1

    def __post_init__(self) -> None:
        rng = np.random.default_rng(self.seed)
        perm = np.arange(256, dtype=int)
        rng.shuffle(perm)
        self.permutation = np.concatenate([perm, perm])

    def _gradient(self, hash_value: np.ndarray) -> np.ndarray:
        gradients = np.array(
            [
                [1, 1, 0],
                [-1, 1, 0],
                [1, -1, 0],
                [-1, -1, 0],
                [1, 0, 1],
                [-1, 0, 1],
                [1, 0, -1],
                [-1, 0, -1],
                [0, 1, 1],
                [0, -1, 1],
                [0, 1, -1],
                [0, -1, -1],
            ]
        )
        return gradients[hash_value % gradients.shape[0]]

    def __call__(self, x: float, y: float, z: float) -> float:
        x_arr = np.asarray(x, dtype=float)
        y_arr = np.asarray(y, dtype=float)
        z_arr = np.asarray(z, dtype=float)

        xi = np.floor(x_arr).astype(int) & 255
        yi = np.floor(y_arr).astype(int) & 255
        zi = np.floor(z_arr).astype(int) & 255
        xf, yf, zf = x_arr - np.floor(x_arr), y_arr - np.floor(y_arr), z_arr - np.floor(z_arr)
        u, v, w = _fade(xf), _fade(yf), _fade(zf)

        def gradient_at(dx: float, dy: float, dz: float, px: int, py: int, pz: int) -> float:
            hash_value = self.permutation[self.permutation[self.permutation[xi + px] + yi + py] + zi + pz]
            gradient = self._gradient(hash_value)
            return float(gradient[..., 0] * dx + gradient[..., 1] * dy + gradient[..., 2] * dz)

        n000 = gradient_at(float(xf), float(yf), float(zf), 0, 0, 0)
        n100 = gradient_at(float(xf - 1), float(yf), float(zf), 1, 0, 0)
        n010 = gradient_at(float(xf), float(yf - 1), float(zf), 0, 1, 0)
        n110 = gradient_at(float(xf - 1), float(yf - 1), float(zf), 1, 1, 0)
        n001 = gradient_at(float(xf), float(yf), float(zf - 1), 0, 0, 1)
        n101 = gradient_at(float(xf - 1), float(yf), float(zf - 1), 1, 0, 1)
        n011 = gradient_at(float(xf), float(yf - 1), float(zf - 1), 0, 1, 1)
        n111 = gradient_at(float(xf - 1), float(yf - 1), float(zf - 1), 1, 1, 1)

        x1 = _lerp(n000, n100, u)
        x2 = _lerp(n010, n110, u)
        x3 = _lerp(n001, n101, u)
        x4 = _lerp(n011, n111, u)
        y1 = _lerp(x1, x2, v)
        y2 = _lerp(x3, x4, v)
        return float(_lerp(y1, y2, w))


## Simulation parameters


In [ ]:
# Centralized simulation controls for quick iteration
simulation_output = Path("notebooks/data/perlin_vector_field_cache.npz")  # File storing precomputed frames

# How long to simulate and how densely to sample time
duration = 8.0  # Seconds of simulated time captured in the cache
frame_rate = 30  # Samples per second written to disk

# Grid layout for the anchored arrows
grid_x_span = (-4.5, 4.5)  # Inclusive horizontal bounds in scene units
grid_y_span = (-2.5, 2.5)  # Inclusive vertical bounds in scene units
grid_x_count = 20  # Number of anchors along the x-axis
grid_y_count = 14  # Number of anchors along the y-axis

# Perlin noise parameters controlling the flow pattern
noise_seed = 7  # Seed for the Perlin sampler (reproducible flow)
noise_scale = 0.6  # Scales scene coordinates into noise space
noise_radius = 0.8  # Radius of the circular path through noise space
noise_speed = 0.9  # Angular speed of the moving noise center (radians/sec)

# Arrow styling
arrow_length = 0.6  # Fixed length applied after normalizing vectors

# Color grading for magnitudes
color_bias = 0.2  # Baseline offset before mapping magnitude to palette index
color_gain = 0.8  # Gain applied to magnitudes before color lookup
palette: list[ManimColor] = cast(
    list[ManimColor],
    color_gradient([BLUE_E, BLUE_C, GREEN_B, YELLOW_C, RED_C], 64),
)  # Gradient used to color arrows by magnitude


## Map grid points into noise space

Each grid anchor is scaled into noise coordinates, offset by a moving center, and sampled to produce a normalized vector and magnitude-driven color.


In [ ]:
noise_sampler = PerlinSampler(seed=noise_seed)

def sample_vector_field(point: np.ndarray, time: float) -> Tuple[np.ndarray, float]:
    px, py = point[:2]
    noise_center = np.array([math.cos(time * noise_speed), math.sin(time * noise_speed), 0.0]) * noise_radius
    domain = np.array([px, py, time * 0.35]) * noise_scale + noise_center
    dx = noise_sampler(*(domain + np.array([0.0, 0.0, 0.5])))
    dy = noise_sampler(*(domain + np.array([2.3, -1.7, -0.25])))
    vector = np.array([dx, dy])
    magnitude = float(np.linalg.norm(vector))
    if magnitude > 1e-6:
        vector = vector / magnitude
    vector *= arrow_length
    return vector, magnitude


def color_from_magnitude(magnitude: float) -> ManimColor:
    scaled = float(np.clip(magnitude * color_gain + color_bias, 0, 1))
    index = int(scaled * (len(palette) - 1))
    return palette[index]


## Stage 1: simulate and cache the field

Sample the Perlin field on a fixed time grid, compute vectors for each anchor, and save the resulting positions and colors to disk. Re-run this cell whenever noise parameters change.

In [ ]:
def run_simulation(output_path: Path = simulation_output) -> Path:
    times = np.linspace(0.0, duration, int(duration * frame_rate) + 1)
    xs = np.linspace(*grid_x_span, grid_x_count)
    ys = np.linspace(*grid_y_span, grid_y_count)
    anchors = np.array([[x, y, 0.0] for x in xs for y in ys], dtype=np.float32)

    starts = np.repeat(anchors[None, ...], len(times), axis=0)
    ends = np.zeros_like(starts)
    magnitudes = np.zeros((len(times), len(anchors)), dtype=np.float32)
    colors = np.empty((len(times), len(anchors)), dtype="<U7")

    for frame_index, t in enumerate(tqdm(times, desc="Simulating frames")):
        for anchor_index, anchor in enumerate(anchors):
            vector, magnitude = sample_vector_field(anchor, float(t))
            ends[frame_index, anchor_index] = anchor + np.append(vector, 0.0)
            magnitudes[frame_index, anchor_index] = magnitude
            colors[frame_index, anchor_index] = color_from_magnitude(magnitude).to_hex()

    output_path.parent.mkdir(parents=True, exist_ok=True)
    np.savez_compressed(
        output_path,
        times=times.astype(np.float32),
        anchors=anchors,
        starts=starts,
        ends=ends,
        magnitudes=magnitudes,
        colors=colors,
        frame_dt=np.float32(times[1] - times[0] if len(times) > 1 else 0.0),
    )
    return output_path

# Uncomment to regenerate the cache when parameters change.
# simulation_path = run_simulation()


## Stage 2: render from the cached data

The Manim scene now reads the precomputed arrow positions and colors. The only animation logic during rendering is frame indexing, so Manim never re-samples the noise while drawing.

In [ ]:
class PerlinNoiseVectorField(Scene):
    def construct(self) -> None:
        if not simulation_output.exists():
            raise FileNotFoundError(f"Run the simulation stage to generate {simulation_output} before rendering.")

        cache = np.load(simulation_output)
        anchors = cache["anchors"]
        starts = cache["starts"]
        ends = cache["ends"]
        colors = cache["colors"]
        frame_dt = float(cache["frame_dt"])
        total_frames = starts.shape[0]

        def make_arrow(anchor: np.ndarray) -> Arrow:
            return Arrow(
                anchor,
                anchor + RIGHT * 0.01,
                buff=0,
                max_stroke_width_to_length_ratio=2.5,
                max_tip_length_to_length_ratio=0.35,
                stroke_width=3.0,
            )

        frame_tracker = ValueTracker(0)

        def update_arrow(arrow: Arrow, anchor_index: int) -> None:
            frame_index = min(int(round(frame_tracker.get_value())), total_frames - 1)
            arrow.put_start_and_end_on(starts[frame_index, anchor_index], ends[frame_index, anchor_index])
            arrow.set_color(ManimColor(colors[frame_index, anchor_index]))

        arrow_list: list[Arrow] = [make_arrow(anchor) for anchor in anchors]
        arrows = VGroup(*arrow_list)

        for index, arrow in enumerate(arrow_list):
            arrow.add_updater(lambda mob, dt, idx=index: update_arrow(cast(Arrow, mob), idx))

        self.play(FadeIn(arrows), run_time=1.5, rate_func=smooth)

        def advance_frame(mob: ValueTracker, dt: float) -> None:
            mob.increment_value(dt / frame_dt)

        frame_tracker.add_updater(advance_frame)
        self.wait(frame_dt * (total_frames - 1))

        for arrow in arrow_list:
            arrow.clear_updaters()
        frame_tracker.clear_updaters()

        self.play(FadeOut(arrows), run_time=1.0, rate_func=smooth)


## Render the scene

Use the Manim IPython magic to generate and display the animation inline. Adjust quality flags as needed.

In [ ]:
%manim -qm -v WARNING PerlinNoiseVectorField